In [1]:
import os
import torch
import cv2
import json
import numpy as np
import matplotlib.pyplot as plt
import torch
from pathlib import Path

In [14]:
class LaneDataset(torch.utils.data.Dataset):
    def __init__(self, dataset_path = "archive/TUSimple/train_set", train= True, image_size = (512,256)):
        self._dataset_path = dataset_path
        self._mode = "train" if train else "eval"
        self._image_size = image_size # w, h

        self._data = []

        if self._mode == 'train':
            label_files = [os.path.join(self._dataset_path, f"label_data_{suffix}.json") for suffix in ("0313", "0531")]
        elif self._mode == 'eval':
            label_files = [os.path.join(self._dataset_path, f"label_data_{suffix}.json") for suffix in ("0601",)]

        
        for label_file in label_files:
             self._preprocess_labels(label_file)

    def __len__(self):
         print(self._data)
         return len(self._data)

    def __getitem__(self, idx):
        image_path = os.path.join(self._dataset_path, self._data[idx][0])
        image = cv2.imread(image_path)
        h , w, c = image.shape
        raw_image = image

        image = cv2.resize(image, self._image_size, interpolation = cv2.INTER_LINEAR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        '''cv2.imshow('image', image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()''' 
        lanes= self._data[idx][1]
        lanes = lanes.reshape(-1,1,2)

        new_image = self._draw(h,w, lanes)
        return image, new_image




    def _draw(self, h, w, lanes):
        image = np.zeros((h, w), dtype=np.uint8)
        for lane in lanes:
            lane = np.array(lane, dtype=np.int32)
            cv2.polylines(image, [lane], False, color=1, thickness=10)

        # Resize the image to the desired size
        image = cv2.resize(image, self._image_size, interpolation=cv2.INTER_NEAREST)

        return image



    def _preprocess_labels(self, labels_path):
            with open(labels_path, 'r') as f:
                for lines in f:
                    labels = json.loads(lines)
                    images = labels['raw_file']
                    lanes = labels['lanes']
                    h_samples = labels['h_samples']
                    lane_coordinates = []
                    for lane in lanes:
                            x = np.array([lane],np.int32).T
                            y = np.array([h_samples], np.int32).T
                            xy = np.hstack((x,y))
                            idx = np.where(xy[:,0]>0)
                            lane_coordinate = xy[idx]
                            lane_coordinates.append(lane_coordinate)
                    self._data.append((images,lane_coordinate))
        
        


In [15]:
data =  LaneDataset()
data.__len__()

[('clips/0313-1/6040/20.jpg', array([[ 781,  270],
       [ 822,  280],
       [ 862,  290],
       [ 903,  300],
       [ 944,  310],
       [ 984,  320],
       [1025,  330],
       [1066,  340],
       [1107,  350],
       [1147,  360],
       [1188,  370],
       [1229,  380],
       [1269,  390]])), ('clips/0313-1/5320/20.jpg', array([[ 780,  270],
       [ 811,  280],
       [ 843,  290],
       [ 875,  300],
       [ 906,  310],
       [ 938,  320],
       [ 970,  330],
       [1001,  340],
       [1033,  350],
       [1065,  360],
       [1096,  370],
       [1128,  380],
       [1160,  390],
       [1191,  400],
       [1223,  410],
       [1255,  420]])), ('clips/0313-1/23700/20.jpg', array([[ 647,  260],
       [ 673,  270],
       [ 698,  280],
       [ 724,  290],
       [ 750,  300],
       [ 775,  310],
       [ 801,  320],
       [ 826,  330],
       [ 852,  340],
       [ 877,  350],
       [ 903,  360],
       [ 928,  370],
       [ 954,  380],
       [ 980,  390],
  

3216

In [16]:
data.__getitem__(2)

(array([[170, 163, 161, ...,  21,  23,  24],
        [174, 174, 177, ...,  23,  23,  23],
        [171, 170, 179, ...,  21,  20,  20],
        ...,
        [ 89,  93,  99, ...,  25,  25,  25],
        [ 88,  97, 111, ...,  22,  22,  22],
        [101, 106, 108, ...,  22,  22,  22]], dtype=uint8),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8))

In [77]:
dataset_path = "archive/TUSimple/train_set"
label_files = [os.path.join(dataset_path,f"label_data_{suffix}.json") for suffix in ("0313", "0531")]
for file in label_files:
    with open(file, 'r') as f:
        for lines in f:
            labels = json.loads(lines)

In [79]:
label_files =['archive\\TUSimple\train_set\\label_data_0313.json',
 'archive\\TUSimple\train_set\\label_data_0531.json']


In [39]:
label_files

['archive\\TUSimple\train_set\\label_data_0313.json',
 'archive\\TUSimple\train_set\\label_data_0531.json']

In [ ]:
class LaneData